In [11]:
import pickle

with open('/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/ignore_outputs/data.pkl', 'rb') as f:
    embeddings, Y, df, X_all, edges_df_list, attributes_df_list = pickle.load(f)

with open('/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/ignore_outputs/test_data.pkl', 'rb') as f:
    test_embeddings, test_Y, test_df, test_X_all, test_edges_df_list, test_attributes_df_list = pickle.load(f)

Y=Y.argmax(1)
test_Y=test_Y.argmax(1)



In [12]:
import pandas as pd
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.data import DataLoader

room_classes_names = ["Background", "Outdoor", "Wall", "Kitchen", "Living Room" ,"Bed Room", "Bath", "Entry", "Railing", "Storage", "Garage"]

for df in attributes_df_list:
    df["Room Type"] = df["Room Type"].apply(lambda x: room_classes_names.index(x))

# Define a function to create a PyTorch Geometric Data object from the room and edge data for each floorplan
def create_data(attributes_df, edges_df):
    edge_index = edges_df[['source', 'target']].values.T
    x = attributes_df[['Area', 'Relative Area', 'Number of neighboring rooms']].values
    y = attributes_df['Room Type'].values
    return Data(x=torch.tensor(x, dtype=torch.float), edge_index=torch.tensor(edge_index, dtype=torch.long), y=torch.tensor(y, dtype=torch.long))

# Create a list of Data objects, one for each floorplan
data_list = []
for i in range(len(attributes_df_list)):
    attributes_df_i = attributes_df_list[i]
    edges_df_i = edges_df_list[i]
    data_list.append(create_data(attributes_df_i, edges_df_i))

# Define the GCN architecture
class Net(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(Net, self).__init__()
        self.conv1 = GCNConv(num_features, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Create the GCN and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(num_features=3, num_classes=len(room_classes_names)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Define the training loop
def train(model, optimizer, data_loader):
    model.train()
    total_loss = 0
    for data in data_loader:
        optimizer.zero_grad()
        out = model(data.x.to(device), data.edge_index.to(device))
        loss = F.nll_loss(out, data.y.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(data_loader.dataset)

# Train the GCN on your floorplan data
num_classes = len(room_classes_names)
model = Net(num_features=3, num_classes=len(room_classes_names)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
data_loader = DataLoader(data_list, batch_size=32, shuffle=True)
for epoch in range(200):
    loss = train(model, optimizer, data_loader)
    print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, loss))

# Define a function to create a PyTorch Geometric Data object from the room and edge data for each floorplan
def create_data(attributes_df, edges_df):
    edge_index = edges_df[['source', 'target']].values.T
    x = attributes_df[['Area', 'Relative Area', 'Number of neighboring rooms']].values
    return Data(x=torch.tensor(x, dtype=torch.float), edge_index=torch.tensor(edge_index, dtype=torch.long), y=None)

# Create a list of Data objects for the test floorplans
test_data_list = []
for i in range(len(test_attributes_df_list)):
    attributes_df_i = test_attributes_df_list[i].drop('Room Type', axis=1)
    edges_df_i = test_edges_df_list[i]
    test_data_list.append(create_data(attributes_df_i, edges_df_i))

# Use the trained model to predict room types for the test data
test_data_loader = DataLoader(test_data_list, batch_size=32, shuffle=False)
model.eval()
predictions = []
with torch.no_grad():
    for data in test_data_loader:
        logits = model(data.x.to(device), data.edge_index.to(device))
        pred = logits.argmax(dim=1)
        pred_room_types = [room_classes_names[i] for i in pred.tolist()]
        # Create a list of None values with the same length as the number of nodes in the graph
        pred_room_types = pred_room_types + [None]*(len(data.x) - len(pred_room_types))
        predictions.extend(pred_room_types)

# Create a list of actual room types for the test data
actual_room_types = []
for attributes_df_i in test_attributes_df_list:
    actual_room_types.extend(attributes_df_i['Room Type'].tolist())

# Compare the actual and predicted room types and calculate accuracy
correct_predictions = 0
for i in range(len(actual_room_types)):
    if actual_room_types[i] == predictions[i]:
        correct_predictions += 1
accuracy = correct_predictions / len(actual_room_types)

print('Accuracy: {:.2f}%'.format(accuracy * 100))



/Users/alishakhan/opt/miniconda3/envs/alishadev/lib/python3.9/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch: 000, Loss: 29215.83359
Epoch: 001, Loss: 22876.69169
Epoch: 002, Loss: 21538.99066
Epoch: 003, Loss: 19912.51577
Epoch: 004, Loss: 15016.15382
Epoch: 005, Loss: 11735.25179
Epoch: 006, Loss: 10323.26438
Epoch: 007, Loss: 6442.07391
Epoch: 008, Loss: 5182.97061
Epoch: 009, Loss: 5950.85146
Epoch: 010, Loss: 6990.53407
Epoch: 011, Loss: 5081.06948
Epoch: 012, Loss: 4734.89062
Epoch: 013, Loss: 4148.22233
Epoch: 014, Loss: 4113.49059
Epoch: 015, Loss: 3060.03217
Epoch: 016, Loss: 3423.81498
Epoch: 017, Loss: 2335.36584
Epoch: 018, Loss: 2810.93115
Epoch: 019, Loss: 2630.17182
Epoch: 020, Loss: 2163.35998
Epoch: 021, Loss: 1962.33078
Epoch: 022, Loss: 1796.84827
Epoch: 023, Loss: 1527.82965
Epoch: 024, Loss: 1530.65859
Epoch: 025, Loss: 1233.69019
Epoch: 026, Loss: 1329.35686
Epoch: 027, Loss: 831.39563
Epoch: 028, Loss: 1260.37792
Epoch: 029, Loss: 1045.37742
Epoch: 030, Loss: 921.35433
Epoch: 031, Loss: 862.30035
Epoch: 032, Loss: 763.69838
Epoch: 033, Loss: 781.73346
Epoch: 034, 

/Users/alishakhan/opt/miniconda3/envs/alishadev/lib/python3.9/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [13]:
from sklearn.metrics import classification_report, confusion_matrix

# Convert the room type labels from strings to integers
room_classes_int = {name: i for i, name in enumerate(room_classes_names)}
actual_room_types_int = [room_classes_int[r] for r in actual_room_types]
predicted_room_types_int = [room_classes_int[r] for r in predictions]

# Get the unique classes in the actual and predicted labels
unique_classes = list(set(actual_room_types_int + predicted_room_types_int))

# Convert the integer labels back to string labels
room_classes_names_int = {i: name for name, i in room_classes_int.items()}
target_names = [room_classes_names_int[i] for i in unique_classes]

# Calculate precision, recall, F1 score, and confusion matrix
print(classification_report(actual_room_types_int, predicted_room_types_int, labels=unique_classes, target_names=target_names))
print(confusion_matrix(actual_room_types_int, predicted_room_types_int, labels=unique_classes))


              precision    recall  f1-score   support

     Kitchen       0.14      0.96      0.25       137
 Living Room       0.00      0.00      0.00       131
    Bed Room       0.00      0.00      0.00       217
        Bath       0.45      0.18      0.25       291
       Entry       0.00      0.00      0.00       174
     Storage       0.00      0.00      0.00        68
      Garage       0.00      0.00      0.00         2

    accuracy                           0.18      1020
   macro avg       0.08      0.16      0.07      1020
weighted avg       0.15      0.18      0.11      1020

[[131   0   0   6   0   0   0]
 [128   0   0   3   0   0   0]
 [203   0   0  14   0   0   0]
 [240   0   0  51   0   0   0]
 [147   0   0  27   0   0   0]
 [ 55   0   0  13   0   0   0]
 [  2   0   0   0   0   0   0]]


/Users/alishakhan/opt/miniconda3/envs/alishadev/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/alishakhan/opt/miniconda3/envs/alishadev/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/alishakhan/opt/miniconda3/envs/alishadev/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf